## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lugovoy,KZ,2022-02-10 17:10:27,42.9472,72.7644,25.03,74,96,5.86,overcast clouds
1,1,Abong Mbang,CM,2022-02-10 17:10:28,3.9833,13.1833,83.57,20,4,1.97,clear sky
2,2,Kulhudhuffushi,MV,2022-02-10 17:10:28,6.6221,73.0700,80.44,77,56,12.39,broken clouds
3,3,Puerto Ayora,EC,2022-02-10 17:10:15,-0.7393,-90.3518,82.36,84,80,5.01,broken clouds
4,4,Tuktoyaktuk,CA,2022-02-10 17:10:28,69.4541,-133.0374,-7.42,98,100,4.97,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Abong Mbang,CM,2022-02-10 17:10:28,3.9833,13.1833,83.57,20,4,1.97,clear sky
2,2,Kulhudhuffushi,MV,2022-02-10 17:10:28,6.6221,73.0700,80.44,77,56,12.39,broken clouds
3,3,Puerto Ayora,EC,2022-02-10 17:10:15,-0.7393,-90.3518,82.36,84,80,5.01,broken clouds
5,5,Busselton,AU,2022-02-10 17:10:29,-33.6500,115.3333,69.08,73,4,7.58,clear sky
8,8,Rikitea,PF,2022-02-10 17:10:30,-23.1203,-134.9692,78.64,74,18,18.01,few clouds
11,11,Souillac,MU,2022-02-10 17:10:31,-20.5167,57.5167,79.18,83,75,3.44,light rain
12,12,Mackay,AU,2022-02-10 17:10:31,-21.1500,149.2000,73.69,70,30,8.55,scattered clouds
14,14,Mount Isa,AU,2022-02-10 17:10:07,-20.7333,139.5000,76.77,65,65,4.61,broken clouds
16,16,Mbini,GQ,2022-02-10 17:10:32,1.5740,9.6134,82.22,74,77,7.18,broken clouds
17,17,Arraial Do Cabo,BR,2022-02-10 17:10:32,-22.9661,-42.0278,71.46,94,100,3.44,mist


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                282
City                   282
Country                279
Date                   282
Lat                    282
Lng                    282
Max Temp               282
Humidity               282
Cloudiness             282
Wind Speed             282
Current Description    282
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_nn_df = preferred_cities_df.dropna()
preferred_cities_nn_df.count()

City_ID                279
City                   279
Country                279
Date                   279
Lat                    279
Lng                    279
Max Temp               279
Humidity               279
Cloudiness             279
Wind Speed             279
Current Description    279
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_nn_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Abong Mbang,CM,83.57,clear sky,3.9833,13.1833,
2,Kulhudhuffushi,MV,80.44,broken clouds,6.6221,73.0700,
3,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
5,Busselton,AU,69.08,clear sky,-33.6500,115.3333,
8,Rikitea,PF,78.64,few clouds,-23.1203,-134.9692,
11,Souillac,MU,79.18,light rain,-20.5167,57.5167,
12,Mackay,AU,73.69,scattered clouds,-21.1500,149.2000,
14,Mount Isa,AU,76.77,broken clouds,-20.7333,139.5000,
16,Mbini,GQ,82.22,broken clouds,1.5740,9.6134,
17,Arraial Do Cabo,BR,71.46,mist,-22.9661,-42.0278,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 24140,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Abong Mbang,CM,83.57,clear sky,3.9833,13.1833,Bagofit Sun City
2,Kulhudhuffushi,MV,80.44,broken clouds,6.6221,73.0700,Asseyri Tourist Inn
3,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Busselton,AU,69.08,clear sky,-33.6500,115.3333,Observatory Guest House
8,Rikitea,PF,78.64,few clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
698,Ananas,HT,75.27,clear sky,19.2466,-71.9557,«Petites Soeurs de l'Incarnation»
699,Ternate,ID,79.97,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
700,Acaponeta,MX,78.84,clear sky,22.4964,-105.3594,Hotel Casa Blanca
701,Kupang,ID,75.85,light rain,-10.1667,123.5833,Hotel On The Rock by Prasanthi


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))